In [1]:
import pylab
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import csv
from bs4 import BeautifulSoup  
from wordcloud import WordCloud,STOPWORDS

import nltk
#nltk.download()  # Download text data sets, including stop words
from nltk.corpus import stopwords # Import the stop word list

import operator
import re

print "initial imports complete"

/Applications/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


initial imports complete


In [2]:
#READ IN THE YELP FILES to a pandas data frame
user = pd.read_csv("yelp_academic_dataset_user.csv", index_col = 'user_id')
print "user read-in complete"
print(user.shape)

business = pd.read_csv("yelp_academic_dataset_business.csv", na_filter=False, index_col = 'business_id')
print "business read-in complete"
print(business.shape)

reviews = pd.read_csv("yelp_academic_dataset_review.csv", index_col = 'review_id')
print "reviews read-in complete"
print(reviews.shape)


user read-in complete
(552339, 22)
business read-in complete
(77445, 97)
reviews read-in complete
(2225213, 9)


In [3]:
return_visit = reviews.duplicated(['user_id', 'business_id'])
print 'return_visit array created'
print(return_visit.shape)

reviews['return_visit'] = return_visit
print 'return_visit array added to reviews'
print(reviews.shape)

#first visit marks reviews which were later followed up
first_visit = reviews.duplicated(['user_id', 'business_id'], keep='last')
print 'first_visit array created'

reviews['first_visit'] = first_visit
print 'first_visit array added to reviews'
print(reviews.shape)

return_visit array created
(2225213,)
return_visit array added to reviews
(2225213, 10)
first_visit array created
first_visit array added to reviews
(2225213, 11)


In [4]:
#print user.head(3)
#print reviews.head(3)
print business['name'][0:5]
print business['categories'][0:5]
print business['review_count'][0:5]
print business['state'][0:5]
print business['city'][0:5]
print business['stars'][0:5]
print business['type'][0:5]

business_id
5UmKMjUEUNdYWqANhGckJw                   Mr Hoagie
UsFtqoBl7naz8AVUBZMjQQ                Clancy's Pub
3eu6MEFlq2Dg7bQh8QbdOg            Joe Cislo's Auto
cE27W9VPgO88Qxe4ol6y_g    Cool Springs Golf Center
HZdLhv6COCleJMo7nPl-RA                     Verizon
Name: name, dtype: object
business_id
5UmKMjUEUNdYWqANhGckJw                         ['Fast Food', 'Restaurants']
UsFtqoBl7naz8AVUBZMjQQ                                        ['Nightlife']
3eu6MEFlq2Dg7bQh8QbdOg                        ['Auto Repair', 'Automotive']
cE27W9VPgO88Qxe4ol6y_g                 ['Active Life', 'Mini Golf', 'Golf']
HZdLhv6COCleJMo7nPl-RA    ['Shopping', 'Home Services', 'Internet Servic...
Name: categories, dtype: object
business_id
5UmKMjUEUNdYWqANhGckJw    4
UsFtqoBl7naz8AVUBZMjQQ    4
3eu6MEFlq2Dg7bQh8QbdOg    3
cE27W9VPgO88Qxe4ol6y_g    5
HZdLhv6COCleJMo7nPl-RA    5
Name: review_count, dtype: int64
business_id
5UmKMjUEUNdYWqANhGckJw    PA
UsFtqoBl7naz8AVUBZMjQQ    PA
3eu6MEFlq2Dg7bQh8QbdOg    PA

In [5]:
business['ncount'] = 1.0
state_count = business.groupby('state').aggregate(sum)
state_count['star_avg'] = state_count['stars'] / state_count['ncount']
#business.groupby('city').aggregate(sum)
state_count

,review_count,open,stars,latitude,longitude,ncount,star_avg
state,,,,,,,
,4,1.0,2.0,4.545614e+01,-7.375287e+01,1.0,2.000000
AL,4,1.0,3.5,3.369801e+01,-1.119247e+02,1.0,3.500000
AR,23,1.0,5.0,3.345076e+01,-1.122743e+02,1.0,5.000000
AZ,913588,28125.0,121011.0,1.092081e+06,-3.652433e+06,32615.0,3.710287
BW,11439,983.0,3963.0,5.135443e+04,8.806647e+03,1048.0,3.781489
CA,126,3.0,10.5,1.178688e+02,-3.038698e+02,3.0,3.500000
EDH,39522,2743.0,12210.5,1.793747e+05,-1.025088e+04,3206.0,3.808640
ELN,59,9.0,45.5,6.712992e+02,-3.675137e+01,12.0,3.791667
FIF,39,5.0,18.5,2.801185e+02,-1.693493e+01,5.0,3.700000


In [6]:
print business[business['state'] == 'BW']['city'][1]
print business[business['state'] == 'EDH']['city'][1]
print business[business['state'] == 'MLN']['city'][1]
print business[business['state'] == 'QC']['city'][1]
print business[business['state'] == 'ON']['city'][1]


Karlsruhe
Edinburgh
Edinburgh
Montréal
Waterloo


In [7]:
from bokeh.io import output_notebook, show
output_notebook()

from bokeh.plotting import figure
#p = figure(plot_width=400, plot_height=400)
#p.square([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], size=[10, 15, 20, 25, 30], color="firebrick", alpha=0.6)
#p.line(x, y, line_width=2)
#show(p) # show the results

from bokeh.charts import Scatter
#p = Scatter(flowers, x='petal_length', y='petal_width', color='species', legend='top_left')
#show(p)

from bokeh.charts import Bar
#p = Bar(autompg, label='yr', values='mpg', agg='median', group='origin', 
#        title="Median MPG by YR, grouped by ORIGIN", legend='top_left', tools='crosshair')
#show(p)

Loading BokehJS ...

In [8]:
#p = figure(plot_width=400, plot_height=400)
#p.square(user.review_count, user.average_stars)
#show(p)

In [9]:
userSMALL = user[user['review_count'] <= 2]
user1 = user[(user['review_count'] > 2) & (user['review_count'] <= 50)]
user2 = user[(user['review_count'] > 50) & (user['review_count'] <= 100)]
user3 = user[(user['review_count'] > 100) & (user['review_count'] <= 500)]
user4 = user[(user['review_count'] > 500) & (user['review_count'] <= 1000)]
userWUT = user[user['review_count'] > 1000]

print 'lengths =', len(userSMALL), len(user1), len(user2), len(user3),  len(user4), len(userWUT)

len_user = len(user)
len_total = len(userSMALL) + len(user1) + len(user2) \
    + len(user3) + len(user4) + len(userWUT)
    
print len_user
print len_total

star_avg0 = userSMALL.average_stars.mean() 
star_avg1 = user1.average_stars.mean() 
star_avg2 = user2.average_stars.mean() 
star_avg3 = user3.average_stars.mean() 
star_avg4 = user4.average_stars.mean()
star_avg5 = userWUT.average_stars.mean()

print star_avg0, star_avg1, star_avg2, star_avg3, star_avg4, star_avg5

lengths = 159935 330496 28092 30533 2681 602
552339
552339
3.69278200519 3.76079280839 3.76705076178 3.75893263027 3.73771353972 3.73735880399


In [10]:
#p = figure(plot_width=400, plot_height=400)
#p.square(user.review_count, user['votes.useful'])
#show(p)

In [11]:
userSMALL = user[user['votes.useful'] <= 1]
user1 = user[(user['votes.useful'] > 1) & (user['votes.useful'] <= 100)]
user2 = user[(user['votes.useful'] > 100) & (user['votes.useful'] <= 1000)]
user3 = user[(user['votes.useful'] > 1000) & (user['votes.useful'] <= 3000)]
user4 = user[(user['votes.useful'] > 3000) & (user['votes.useful'] <= 5000)]
userWUT = user[user['votes.useful'] > 5000]

print 'lengths =', len(userSMALL), len(user1), len(user2), len(user3),  len(user4), len(userWUT)

len_user = len(user)
len_total = len(userSMALL) + len(user1) + len(user2) \
    + len(user3) + len(user4) + len(userWUT)
    
print len_user
print len_total

star_avg0 = userSMALL.average_stars.mean() 
star_avg1 = user1.average_stars.mean() 
star_avg2 = user2.average_stars.mean() 
star_avg3 = user3.average_stars.mean() 
star_avg4 = user4.average_stars.mean()
star_avg5 = userWUT.average_stars.mean()

print star_avg0, star_avg1, star_avg2, star_avg3, star_avg4, star_avg5

lengths = 178806 331580 37722 3299 483 449
552339
552339
3.84474805096 3.68629078955 3.72902470707 3.76250378903 3.79954451346 3.83069042316


In [12]:
print user['votes.cool'].min()
print user['votes.cool'].max()

userSMALL = user[user['votes.cool'] <= 1]
user1 = user[(user['votes.cool'] > 1) & (user['votes.cool'] <= 10)]
user2 = user[(user['votes.cool'] > 10) & (user['votes.cool'] <= 1000)]
user3 = user[(user['votes.cool'] > 1000) & (user['votes.cool'] <= 3000)]
user4 = user[(user['votes.cool'] > 3000) & (user['votes.cool'] <= 5000)]
userWUT = user[user['votes.cool'] > 5000]

print 'lengths =', len(userSMALL), len(user1), len(user2), len(user3),  len(user4), len(userWUT)

len_user = len(user)
len_total = len(userSMALL) + len(user1) + len(user2) \
    + len(user3) + len(user4) + len(userWUT)
    
print len_user
print len_total

star_avg0 = userSMALL.average_stars.mean() 
star_avg1 = user1.average_stars.mean() 
star_avg2 = user2.average_stars.mean() 
star_avg3 = user3.average_stars.mean() 
star_avg4 = user4.average_stars.mean()
star_avg5 = userWUT.average_stars.mean()

print star_avg0, star_avg1, star_avg2, star_avg3, star_avg4, star_avg5

0
52441
lengths = 314665 148265 86965 1812 308 324
552339
552339
3.68076579855 3.83285583246 3.80122911516 3.81642384106 3.82733766234 3.83839506173


In [13]:
user['friends2'] = user['friends'].str.split(',') 
#print user['friends2'].iloc[10], len(user['friends2'].iloc[10])

user['nfriends'] = 0
print len(user['nfriends'])

pd.options.mode.chained_assignment = None  # default='warn'
for i in range(0, len_user-1):
    user['nfriends'].iloc[i] = len(user['friends2'].iloc[i])
    if (i%10000 == 0):
        print 'i =', i, user['nfriends'].iloc[i], len(user['friends2'].iloc[i])

#user['nfriends'] = user['nfriends'].apply(lambda x: len(user['friends2']))

print user['nfriends'].min()
print user['nfriends'].max()
    

552339
i = 0 202 202
i = 10000 1 1
i = 20000 1 1
i = 30000 1 1
i = 40000 1 1
i = 50000 1 1
i = 60000 1 1
i = 70000 1 1
i = 80000 1 1
i = 90000 1 1
i = 100000 5 5
i = 110000 1 1
i = 120000 1 1
i = 130000 4 4
i = 140000 6 6
i = 150000 1 1
i = 160000 1 1
i = 170000 4 4
i = 180000 14 14
i = 190000 1 1
i = 200000 1 1
i = 210000 1 1
i = 220000 1 1
i = 230000 1 1
i = 240000 1 1
i = 250000 1 1
i = 260000 83 83
i = 270000 1 1
i = 280000 1 1
i = 290000 16 16
i = 300000 1 1
i = 310000 1 1
i = 320000 3 3
i = 330000 1 1
i = 340000 12 12
i = 350000 6 6
i = 360000 1 1
i = 370000 1 1
i = 380000 1 1
i = 390000 1 1
i = 400000 6 6
i = 410000 1 1
i = 420000 1 1
i = 430000 1 1
i = 440000 1 1
i = 450000 24 24
i = 460000 1 1
i = 470000 6 6
i = 480000 1 1
i = 490000 1 1
i = 500000 20 20
i = 510000 42 42
i = 520000 30 30
i = 530000 1 1
i = 540000 9 9
i = 550000 1 1
0
3812


In [14]:
userSMALL = user[user['nfriends'] == 1]
user1 = user[(user['nfriends'] > 1) & (user['nfriends'] <= 5)]
user2 = user[(user['nfriends'] > 5) & (user['nfriends'] <= 10)]
user3 = user[(user['nfriends'] > 10) & (user['nfriends'] <= 100)]
userWUT = user[user['nfriends'] > 100]

print 'lengths =', len(userSMALL), len(user1), len(user2), len(user3),  len(userWUT)

len_user = len(user)
len_total = len(userSMALL) + len(user1) + len(user2) \
    + len(user3) + len(userWUT)
    
print len_user
print len_total

star_avg0 = userSMALL.average_stars.mean() 
star_avg1 = user1.average_stars.mean() 
star_avg2 = user2.average_stars.mean() 
star_avg3 = user3.average_stars.mean() 
star_avg4 = userWUT.average_stars.mean()

print star_avg0, star_avg1, star_avg2, star_avg3, star_avg4

lengths = 369201 87498 35244 54982 5413
552339
552338
3.67455735494 3.85017874694 3.89541936216 3.90185096941 3.88663033438


In [15]:
user['elite2'] = user['elite'].str.split(',') 
print user['elite2'].iloc[2], len(user['elite2'].iloc[2])

user['nelite'] = 0
len_user = len(user)
print 'len_user =', len(user['nelite'])
for i in range(0, len_user-1):
    user['nelite'].iloc[i] = len(user['elite2'].iloc[i])
    if (i%10000 == 0):
        print 'i =', i, user['nelite'].iloc[i], len(user['elite2'].iloc[i])

print user['nelite'].min()
print user['nelite'].max()

['[2005', ' 2006', ' 2007', ' 2008', ' 2009', ' 2010', ' 2011', ' 2012', ' 2013', ' 2014', ' 2015]'] 11
len_user = 552339
i = 0 2 2
i = 10000 1 1
i = 20000 1 1
i = 30000 1 1
i = 40000 1 1
i = 50000 1 1
i = 60000 1 1
i = 70000 1 1
i = 80000 1 1
i = 90000 1 1
i = 100000 1 1
i = 110000 1 1
i = 120000 1 1
i = 130000 1 1
i = 140000 1 1
i = 150000 1 1
i = 160000 1 1
i = 170000 1 1
i = 180000 4 4
i = 190000 1 1
i = 200000 1 1
i = 210000 1 1
i = 220000 1 1
i = 230000 1 1
i = 240000 1 1
i = 250000 1 1
i = 260000 5 5
i = 270000 1 1
i = 280000 1 1
i = 290000 1 1
i = 300000 1 1
i = 310000 1 1
i = 320000 1 1
i = 330000 1 1
i = 340000 1 1
i = 350000 1 1
i = 360000 1 1
i = 370000 1 1
i = 380000 1 1
i = 390000 1 1
i = 400000 1 1
i = 410000 1 1
i = 420000 1 1
i = 430000 1 1
i = 440000 1 1
i = 450000 3 3
i = 460000 1 1
i = 470000 1 1
i = 480000 1 1
i = 490000 1 1
i = 500000 1 1
i = 510000 5 5
i = 520000 1 1
i = 530000 1 1
i = 540000 1 1
i = 550000 1 1
0
11


In [16]:
userSMALL = user[user['nelite'] == 1]
user1 = user[user['nelite'] > 1]
#user1 = user[(user['nelite'] > 1) & (user['nelite'] <= 3)]
#user2 = user[(user['nelite'] > 3) & (user['nelite'] <= 5)]
#user3 = user[(user['nelite'] > 5) & (user['nelite'] <= 8)]
#userWUT = user[user['nelite'] > 8]

print 'lengths =', len(userSMALL), len(user1)

len_user = len(user)
len_total = len(userSMALL) + len(user1) 

print len_user
print len_total

star_avg0 = userSMALL.average_stars.mean() 
star_avg1 = user1.average_stars.mean() 

print star_avg0, star_avg1

lengths = 529635 22703
552339
552338
3.73915462535 3.78831387922


In [17]:
import datetime
from datetime import datetime

#reviews['date'] = reviews['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').date())
reviews['date'] = pd.to_datetime(reviews['date'])
reviews['year'], reviews['month'], reviews['day'] = reviews['date'].dt.year, reviews['date'].dt.month, reviews['date'].dt.day

pd.options.mode.chained_assignment = None  # default='warn'
#for i in range(0,len(reviews)):
#    reviews['date_new'][i] = datetime.strptime(reviews['date'][i], '%Y-%m-%d').date()
#    if i%1000 == 0:
#        print "i =", i
    


In [18]:
#get average star rating for every year
import datetime
import string
from datetime import date

print reviews['date'][1:10]
    
years = []
y = 2005

for i in range(0,11):
    years.append(y+i)

print years
    
for y in years:
    start_date = datetime.date(year=y,month=1,day=1)
    end_date = datetime.date(year=y+1,month=1,day=1)
    subset = reviews[(reviews['date'] > start_date) & (reviews['date'] < end_date) ]
    print y, len(subset), subset['stars'].mean()


review_id
KPvLNJ21_4wbYNctrOwWdQ   2014-02-13
fFSoGV46Yxuwbr3fHNuZig   2015-10-31
Di3exaUCFNw1V4kSNW5pgA   2013-11-08
0Lua2-PbqEQMjD9r89-asw   2014-03-29
7N9j5YbBHBW6qguE5DAeyA   2014-10-29
mjCJR33jvUNt41iJCxDU_g   2014-11-28
Ieh3kfZ-5J9pLju4JiQDvQ   2014-02-27
PU28OoBSHpZLkYGCmNxlmg   2015-06-16
XsA6AojkWjOHA4FmuAb8XQ   2012-08-19
Name: date, dtype: datetime64[ns]
[2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
2005 690 4.03043478261
2006 4178 3.81019626616
2007 17553 3.82179684384
2008 44863 3.72128480039
2009 72581 3.69752414544
2010 137840 3.73993035403
2011 210767 3.74245019382
2012 246410 3.73217807719
2013 336433 3.748802882
2014 488446 3.75978920904
2015 660681 3.78084885141


In [19]:
import datetime
from datetime import datetime

for i in range(1,13):
    subset = reviews[(reviews['month']==i)]
    print i, len(subset), subset['stars'].mean()

1 170941 3.7543713913
2 154833 3.75327611039
3 181417 3.74027241108
4 176704 3.74996604491
5 183310 3.74702416671
6 190490 3.76055435981
7 213509 3.7739954756
8 216273 3.76707679646
9 189777 3.7589486608
10 191808 3.76248644478
11 183120 3.75922892093
12 173031 3.73848616722


In [20]:
user['user_id'] = user.index
rev_and_user = pd.merge(reviews, user, how='inner', on=['user_id', 'user_id'])
print rev_and_user.shape
rev_and_user.head(3)

(2225213, 40)


,user_id,text,votes.cool_x,business_id,votes.funny_x,stars,date,type_x,votes.useful_x,return_visit,...,votes.cool_y,compliments.list,votes.funny_y,compliments.photos,compliments.funny,votes.useful_y,friends2,nfriends,elite2,nelite
0,PUFPaY9KxDAcGqfsorJp3Q,"Mr Hoagie is an institution. Walking in, it do...",0,5UmKMjUEUNdYWqANhGckJw,0,4,2012-08-01,review,0,False,...,12,NaN,25,NaN,NaN,166,[[]],1,[[]],1
1,PUFPaY9KxDAcGqfsorJp3Q,It is now a shop and save.,0,WaHXyBwljbKNPmmJZn5j8Q,0,3,2012-08-01,review,0,False,...,12,NaN,25,NaN,NaN,166,[[]],1,[[]],1
2,PUFPaY9KxDAcGqfsorJp3Q,"I have to admit, the first time I hate here, I...",0,u22QDAON1kk0wjoTliyftw,0,4,2012-08-05,review,4,False,...,12,NaN,25,NaN,NaN,166,[[]],1,[[]],1


In [21]:
rev_and_user.to_pickle('rev_and_user')

In [22]:
print rev_and_user['first_visit'][1:5]
print rev_and_user['nfriends'][1:5]

1    False
2    False
3    False
4    False
Name: first_visit, dtype: bool
1    1
2    1
3    1
4    1
Name: nfriends, dtype: int64
